# Risk Management

## Overview
Risk management is the process of identifying, measuring, and mitigating financial risks. It aims to protect portfolios from adverse market movements while optimizing risk-adjusted returns.

## Types of Financial Risk

| Risk Type | Description | Example |
|-----------|-------------|----------|
| **Market Risk** | Price movements in equities, rates, FX, commodities | Stock market crash |
| **Credit Risk** | Counterparty default or downgrade | Bond issuer bankruptcy |
| **Liquidity Risk** | Inability to trade without price impact | Illiquid bond during crisis |
| **Operational Risk** | System failures, fraud, human error | Trading system outage |
| **Model Risk** | Incorrect model assumptions | VaR underestimation |

## Value at Risk (VaR)

VaR answers: *"What is the maximum loss over a given period with a specified confidence level?"*

$$\text{VaR}_{\alpha} = -\inf\{x : P(L \leq x) \geq 1 - \alpha\}$$

**Example:** A 1-day 99% VaR of $1M means there's a 1% chance of losing more than $1M in one day.

### VaR Methods

1. **Parametric (Variance-Covariance)**
   $$\text{VaR} = \mu + z_\alpha \cdot \sigma$$
   - Assumes normal distribution
   - Fast computation

2. **Historical Simulation**
   - Uses actual historical returns
   - No distributional assumptions

3. **Monte Carlo Simulation**
   - Generates random scenarios
   - Most flexible, computationally intensive

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

np.random.seed(42)

# Simulate portfolio returns
returns = np.random.normal(0.0005, 0.02, 1000)  # Daily returns

# Calculate VaR at different confidence levels
confidence_levels = [0.90, 0.95, 0.99]
vars = {conf: np.percentile(returns, (1-conf)*100) for conf in confidence_levels}

fig = go.Figure()

# Histogram of returns
fig.add_trace(go.Histogram(
    x=returns, nbinsx=50, name='Returns Distribution',
    marker_color='rgba(52, 152, 219, 0.7)'
))

# VaR lines
colors = ['#2ecc71', '#f39c12', '#e74c3c']
for (conf, var), color in zip(vars.items(), colors):
    fig.add_vline(x=var, line_dash='dash', line_color=color,
                  annotation_text=f'{int(conf*100)}% VaR: {var*100:.2f}%')

fig.update_layout(
    title='Portfolio Return Distribution with VaR',
    xaxis_title='Daily Return', yaxis_title='Frequency',
    template='plotly_white'
)
fig.show()

print("VaR Summary (Portfolio Value = $10M):")
for conf, var in vars.items():
    print(f"  {int(conf*100)}% VaR: ${abs(var)*10_000_000:,.0f}")

## Conditional VaR (Expected Shortfall)

CVaR (or Expected Shortfall) measures the **expected loss given that the loss exceeds VaR**:

$$\text{CVaR}_\alpha = E[L | L > \text{VaR}_\alpha]$$

**Advantages over VaR:**
- Considers tail severity, not just probability
- Coherent risk measure (satisfies subadditivity)
- Better captures extreme event risk

In [ ]:
# VaR vs CVaR comparison
def calculate_var_cvar(returns, confidence=0.95):
    var = np.percentile(returns, (1-confidence)*100)
    cvar = returns[returns <= var].mean()
    return var, cvar

# Normal vs Fat-tailed distribution
normal_returns = np.random.normal(0, 0.02, 10000)
fat_tail_returns = np.random.standard_t(df=3, size=10000) * 0.015  # t-distribution

fig = make_subplots(rows=1, cols=2, subplot_titles=(
    'Normal Distribution', 'Fat-Tailed Distribution (t-dist, df=3)'
))

for idx, (returns_data, name) in enumerate([(normal_returns, 'Normal'), (fat_tail_returns, 'Fat-tailed')]):
    var, cvar = calculate_var_cvar(returns_data, 0.95)
    
    fig.add_trace(go.Histogram(
        x=returns_data, nbinsx=100, name=name,
        marker_color='rgba(52, 152, 219, 0.6)'
    ), row=1, col=idx+1)
    
    fig.add_vline(x=var, line_dash='dash', line_color='#f39c12', row=1, col=idx+1)
    fig.add_vline(x=cvar, line_dash='solid', line_color='#e74c3c', row=1, col=idx+1)

fig.update_layout(title='VaR (orange) vs CVaR (red) Comparison', template='plotly_white', showlegend=False)
fig.show()

print("95% Confidence Metrics:")
print(f"  Normal:     VaR={calculate_var_cvar(normal_returns)[0]*100:.2f}%, CVaR={calculate_var_cvar(normal_returns)[1]*100:.2f}%")
print(f"  Fat-tailed: VaR={calculate_var_cvar(fat_tail_returns)[0]*100:.2f}%, CVaR={calculate_var_cvar(fat_tail_returns)[1]*100:.2f}%")

## Volatility Models

### Historical Volatility
$$\sigma = \sqrt{\frac{252}{n-1} \sum_{i=1}^{n} (r_i - \bar{r})^2}$$

### EWMA (Exponentially Weighted Moving Average)
$$\sigma_t^2 = \lambda \sigma_{t-1}^2 + (1-\lambda) r_{t-1}^2$$

RiskMetrics uses $\lambda = 0.94$ for daily data.

### GARCH(1,1)
$$\sigma_t^2 = \omega + \alpha r_{t-1}^2 + \beta \sigma_{t-1}^2$$

Captures volatility clustering (high volatility tends to follow high volatility).

In [ ]:
# EWMA volatility estimation
returns = np.random.normal(0, 0.02, 500)
# Add volatility clustering
for i in range(100, 150):
    returns[i] *= 2  # High volatility period

lambda_param = 0.94
ewma_var = np.zeros(len(returns))
ewma_var[0] = returns[0]**2

for i in range(1, len(returns)):
    ewma_var[i] = lambda_param * ewma_var[i-1] + (1 - lambda_param) * returns[i-1]**2

ewma_vol = np.sqrt(ewma_var) * np.sqrt(252)  # Annualized

# Rolling window volatility for comparison
window = 20
rolling_vol = np.array([np.std(returns[max(0,i-window):i]) * np.sqrt(252) if i > 0 else 0 for i in range(len(returns))])

fig = make_subplots(rows=2, cols=1, subplot_titles=('Returns', 'Volatility Estimates'))

fig.add_trace(go.Scatter(y=returns, mode='lines', name='Returns', line=dict(color='#3498db')), row=1, col=1)
fig.add_trace(go.Scatter(y=ewma_vol, mode='lines', name='EWMA Vol', line=dict(color='#e74c3c')), row=2, col=1)
fig.add_trace(go.Scatter(y=rolling_vol, mode='lines', name='Rolling Vol', line=dict(color='#2ecc71', dash='dash')), row=2, col=1)

fig.update_layout(title='EWMA vs Rolling Volatility', template='plotly_white')
fig

## Stress Testing and Scenario Analysis

### Historical Scenarios
- 1987 Black Monday (-22.6%)
- 2008 Financial Crisis
- 2020 COVID Crash

### Hypothetical Scenarios
- Interest rate shock (+/- 200 bps)
- Currency devaluation
- Commodity price spike

## Risk-Adjusted Performance Metrics

| Metric | Formula | Interpretation |
|--------|---------|----------------|
| **Sharpe Ratio** | $\frac{R_p - R_f}{\sigma_p}$ | Return per unit of total risk |
| **Sortino Ratio** | $\frac{R_p - R_f}{\sigma_{downside}}$ | Return per unit of downside risk |
| **Information Ratio** | $\frac{R_p - R_b}{\sigma_{tracking}}$ | Active return per tracking error |
| **Max Drawdown** | $\max_t \left(\frac{\text{Peak}_t - \text{Trough}_t}{\text{Peak}_t}\right)$ | Largest peak-to-trough decline |

## Key Takeaways

1. **VaR has limitations** — Doesn't capture tail severity; use CVaR for coherent measure
2. **Fat tails matter** — Financial returns are not normally distributed
3. **Volatility clusters** — Use EWMA/GARCH for dynamic volatility estimation
4. **Diversification reduces risk** — But correlations increase during crises
5. **Stress test regularly** — Historical scenarios reveal portfolio vulnerabilities